In [ ]:
import cv2 as cv
import mediapipe as mp
import numpy as np

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic


In [ ]:

cap = cv.VideoCapture(0)

collected_data = []

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for label in range(ord('A'), ord('Z')+1):
        label_char = chr(label)
        
        for _ in range(10):
            print(f"Collecting data for label: {label_char}")

            while cap.isOpened():
                ret, frame = cap.read()

                if not ret:
                    print("Failed to capture frame")
                    break

                img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                results = holistic.process(img)

                img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(img, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                           mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                           mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))

                mp_drawing.draw_landmarks(img, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                           mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                           mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))

                cv.imshow('Video Capture', img)

                key = cv.waitKey(10)
                if key == ord(label_char):
                    hand_landmarks = results.right_hand_landmarks
                    collected_data.append((hand_landmarks, label_char))

                elif key == ord('d'):
                    break

    for label in range(48, 58): 
        label_char = chr(label)
        
        for _ in range(10):
            print(f"Collecting data for label: {label_char}")

            while cap.isOpened():
                ret, frame = cap.read()

                if not ret:
                    print("Failed to capture frame")
                    break

                img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                results = holistic.process(img)

                img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(img, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                           mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                           mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))

                mp_drawing.draw_landmarks(img, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                           mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                           mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))

                cv.imshow('Video Capture', img)

                key = cv.waitKey(10)
                if key == ord(label_char):
                    hand_landmarks = results.right_hand_landmarks
                    collected_data.append((hand_landmarks, label_char))

                elif key == ord('d'):
                    break

    np.save('collected_data.npy', collected_data)

cap.release()
cv.destroyAllWindows()


In [ ]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

data = np.load('collected_data.npy', allow_pickle=True)


X = [data[0] for data in collected_data]
y = [data[1] for data in collected_data]


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)

X_train_flat = [np.concatenate(hand_landmarks) for hand_landmarks in X_train]
X_test_flat = [np.concatenate(hand_landmarks) for hand_landmarks in X_test]

model = RandomForestClassifier(random_state=42)
model.fit(X_train_flat, y_train)

y_pred = model.predict(X_test_flat)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)
